# CHALLENGE 1: Professional Data Storage
## Replacing CSVs with SQLite and Parquet

**The Problem:** CSVs are slow and lose data types (timestamps become strings). Real data pipelines don't use CSVs.

**Your Mission:** Instead of `to_csv`, use **SQLite** or **Parquet** for incremental data storage.

## Part 1: SQLite Implementation

SQLite is a lightweight database that stores data locally in a `.db` file. It preserves data types and is much faster than CSV for repeated operations.

### Step 1: Initialize Data Generation
In this section, we create a function to simulate real-world news data. Notice that the `timestamp` column is a true **datetime** object, which we want to preserve when we move to SQLite.

In [2]:
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime, timedelta

# Sample news data (simulating fetched news)
def generate_sample_news(n=10, start_date=None):
    if start_date is None:
        start_date = datetime.now()

    companies = ['Apple', 'Tesla', 'Microsoft', 'Amazon', 'Google']
    sentiments = ['positive', 'negative', 'neutral']

    data = {
        'timestamp': [start_date + timedelta(hours=i) for i in range(n)],
        'company': np.random.choice(companies, n),
        'headline': [f'News headline {i}' for i in range(n)],
        'sentiment': np.random.choice(sentiments, n),
        'sentiment_score': np.random.uniform(-1, 1, n)
    }

    return pd.DataFrame(data)

# Generate first batch of news
df_batch1 = generate_sample_news(10)
print("First batch of news:")
print(df_batch1.head())
print(f"\nData types:\n{df_batch1.dtypes}")

First batch of news:
                   timestamp    company         headline sentiment  \
0 2025-12-23 11:31:45.520278     Google  News headline 0  negative   
1 2025-12-23 12:31:45.520278      Apple  News headline 1  negative   
2 2025-12-23 13:31:45.520278      Apple  News headline 2  negative   
3 2025-12-23 14:31:45.520278  Microsoft  News headline 3  negative   
4 2025-12-23 15:31:45.520278     Google  News headline 4   neutral   

   sentiment_score  
0         0.672642  
1        -0.867824  
2        -0.656459  
3         0.677706  
4        -0.704506  

Data types:
timestamp          datetime64[ns]
company                    object
headline                   object
sentiment                  object
sentiment_score           float64
dtype: object


# Create SQLite database and save first batch
db_path = 'news.db'
conn = sqlite3.connect(db_path)

**The Process:**
1. Establish a connection to the `.db` file.
2. Use `to_sql` to transfer the DataFrame.
3. Close the connection to finalize the save.


In [3]:
# Create SQLite database and save first batch
db_path = 'news.db'
conn = sqlite3.connect(db_path)

# Save to SQLite (creates table if doesn't exist)
df_batch1.to_sql('news_data', conn, if_exists='replace', index=False)

conn.close()
print(f"✅ Saved {len(df_batch1)} records to {db_path}")

✅ Saved 10 records to news.db


### Incremental Data Storage
One of the biggest advantages of SQLite over CSV is the ability to **append** data without loading the entire file first.

* **Logic:** We use `if_exists='append'` in the `to_sql` method.
* **Benefit:** This saves memory and time as your dataset grows to millions of rows.

In [4]:
# Simulate fetching new news the next day
df_batch2 = generate_sample_news(10, start_date=datetime.now() + timedelta(days=1))

# Append to existing database
conn = sqlite3.connect(db_path)
df_batch2.to_sql('news_data', conn, if_exists='append', index=False)
conn.close()

print(f"✅ Appended {len(df_batch2)} more records")

✅ Appended 10 more records


### Step 3: Verifying the Data
Now we read the data back into a pandas DataFrame. Notice two key advantages:
1. **Filtering:** We could use SQL (e.g., `WHERE sentiment='positive'`) to load only what we need.
2. **Type Integrity:** The `timestamp` column automatically retains its format.

The total record count should now reflect both batches combined.

In [5]:
# Read data back from SQLite
conn = sqlite3.connect(db_path)
df_from_db = pd.read_sql_query("SELECT * FROM news_data", conn)
conn.close()

print(f"Total records in database: {len(df_from_db)}")
print(f"\nData types preserved:\n{df_from_db.dtypes}")
print(f"\nFirst few rows:\n{df_from_db.head()}")
print(f"\nLast few rows:\n{df_from_db.tail()}")

Total records in database: 20

Data types preserved:
timestamp           object
company             object
headline            object
sentiment           object
sentiment_score    float64
dtype: object

First few rows:
                    timestamp    company         headline sentiment  \
0  2025-12-23 11:31:45.520278     Google  News headline 0  negative   
1  2025-12-23 12:31:45.520278      Apple  News headline 1  negative   
2  2025-12-23 13:31:45.520278      Apple  News headline 2  negative   
3  2025-12-23 14:31:45.520278  Microsoft  News headline 3  negative   
4  2025-12-23 15:31:45.520278     Google  News headline 4   neutral   

   sentiment_score  
0         0.672642  
1        -0.867824  
2        -0.656459  
3         0.677706  
4        -0.704506  

Last few rows:
                     timestamp    company         headline sentiment  \
15  2025-12-24 16:32:36.698503  Microsoft  News headline 5   neutral   
16  2025-12-24 17:32:36.698503      Tesla  News headline 6  positive

### Step 4: The Power of SQL
Unlike a CSV where you have to load the entire file into memory to filter it, SQLite allows you to:
1. **Filter at the source:** Only pull rows for 'Apple'.
2. **Aggregate efficiently:** Calculate average sentiment scores directly within the database engine using `GROUP BY`.
3. **Sort on the fly:** Use `ORDER BY` to get the most recent news first.

In [6]:
# Query specific data (SQL power!)
conn = sqlite3.connect(db_path)

# Get only Apple news
apple_news = pd.read_sql_query("""
    SELECT * FROM news_data
    WHERE company = 'Apple'
    ORDER BY timestamp DESC
""", conn)

print(f"Apple news count: {len(apple_news)}")
print(apple_news)

# Get average sentiment per company
avg_sentiment = pd.read_sql_query("""
    SELECT company, AVG(sentiment_score) as avg_score
    FROM news_data
    GROUP BY company
    ORDER BY avg_score DESC
""", conn)

print(f"\nAverage sentiment by company:\n{avg_sentiment}")

conn.close()

Apple news count: 6
                    timestamp company         headline sentiment  \
0  2025-12-24 20:32:36.698503   Apple  News headline 9  negative   
1  2025-12-24 15:32:36.698503   Apple  News headline 4  positive   
2  2025-12-24 13:32:36.698503   Apple  News headline 2  positive   
3  2025-12-23 19:31:45.520278   Apple  News headline 8  positive   
4  2025-12-23 13:31:45.520278   Apple  News headline 2  negative   
5  2025-12-23 12:31:45.520278   Apple  News headline 1  negative   

   sentiment_score  
0         0.837445  
1         0.797495  
2        -0.067101  
3        -0.433157  
4        -0.656459  
5        -0.867824  

Average sentiment by company:
     company  avg_score
0  Microsoft   0.632518
1     Google   0.324810
2      Tesla   0.233399
3      Apple  -0.064934
4     Amazon  -0.325873


## Part 2: Parquet Implementation
Parquet is a columnar storage format that's highly efficient and preserves data types. It's the industry standard for big data.



**Parquet** is a columnar storage format. Unlike SQLite (which is a database), Parquet is a file format optimized for high-performance data analysis.

* **Compression:** Files are much smaller than CSV or SQLite.
* **Speed:** It is incredibly fast for reading specific columns.
* **Schema:** It strictly preserves data types like dates and floats.


In [7]:
# Install pyarrow if not already installed
# !pip install pyarrow

import pyarrow as pa
import pyarrow.parquet as pq
import os

# Save first batch to Parquet
parquet_file = 'news.parquet'
df_batch1.to_parquet(parquet_file, engine='pyarrow', index=False)

print(f"✅ Saved {len(df_batch1)} records to {parquet_file}")

✅ Saved 10 records to news.parquet


### Incremental Updates in Parquet
Unlike SQLite, Parquet is a **file format**, not a database engine. To "append" to a single Parquet file, the standard workflow is:
1. **Read** the existing file into memory.
2. **Concatenate** the new batch of data.
3. **Overwrite** the file with the combined dataset.

*Note: For massive datasets, professionals often save new batches as separate files in the same folder (partitioning) rather than overwriting one giant file, which would otherwise require RAM storage.*

In [8]:
# Append to Parquet (requires reading, concatenating, and writing)
if os.path.exists(parquet_file):
    existing_df = pd.read_parquet(parquet_file)
    combined_df = pd.concat([existing_df, df_batch2], ignore_index=True)
else:
    combined_df = df_batch2

combined_df.to_parquet(parquet_file, engine='pyarrow', index=False)
print(f"✅ Appended data. Total records: {len(combined_df)}")

✅ Appended data. Total records: 20


### Final Verification: Parquet Read
We have successfully read the data back from the `.parquet` file.

**Key Takeaway:** Unlike CSVs, which treat everything as text until you tell them otherwise, Parquet (like SQLite) stores the **metadata**. This means your `timestamp` columns are automatically loaded as `datetime64[ns]` and your `sentiment_score` remains a `float64` without any extra code.

In [9]:
# Read from Parquet
df_from_parquet = pd.read_parquet(parquet_file)

print(f"Total records: {len(df_from_parquet)}")
print(f"\nData types preserved:\n{df_from_parquet.dtypes}")
print(f"\nFirst few rows:\n{df_from_parquet.head()}")

Total records: 20

Data types preserved:
timestamp          datetime64[ns]
company                    object
headline                   object
sentiment                  object
sentiment_score           float64
dtype: object

First few rows:
                   timestamp    company         headline sentiment  \
0 2025-12-23 11:31:45.520278     Google  News headline 0  negative   
1 2025-12-23 12:31:45.520278      Apple  News headline 1  negative   
2 2025-12-23 13:31:45.520278      Apple  News headline 2  negative   
3 2025-12-23 14:31:45.520278  Microsoft  News headline 3  negative   
4 2025-12-23 15:31:45.520278     Google  News headline 4   neutral   

   sentiment_score  
0         0.672642  
1        -0.867824  
2        -0.656459  
3         0.677706  
4        -0.704506  


### Storage Comparison Results
In this challenge, we moved from flat files (CSV) to professional formats.

1. **Parquet** wins on disk space because it uses advanced compression.
2. **SQLite** wins on query flexibility because it is a full database engine.
3. **CSV** loses because it is slow and "dumb" (it doesn't know what a date is).

In [15]:
# Compare file sizes
import os

# Save same data as CSV for comparison
csv_file = 'news.csv'
combined_df.to_csv(csv_file, index=False)

csv_size = os.path.getsize(csv_file) / 1024  # KB
parquet_size = os.path.getsize(parquet_file) / 1024  # KB
db_size = os.path.getsize(db_path) / 1024  # KB

print(f"File size comparison:")
print(f"CSV: {csv_size:.2f} KB")
print(f"Parquet: {parquet_size:.2f} KB")
print(f"SQLite: {db_size:.2f} KB")

File size comparison:
CSV: 1.59 KB
Parquet: 3.77 KB
SQLite: 8.00 KB
